In [ ]:
#you'll need to pip install a few things (again this is in linux but it shouldn't be too hard to install these packages in mac)
# !pip install mido
# !pip install midi2audio
# !sudo apt-get install fluidsynth

In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage, bpm2tempo, tempo2bpm, second2tick, tick2second
from midi2audio import FluidSynth
from IPython.display import Audio
import os

In [ ]:
#this generates the note name from the midi note number (aka note 60 is C4, 61 is C#4, etc.)

NOTE_NAMES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

def get_note_name(midi_note):
    """Get the note name from a MIDI note number."""
    return NOTE_NAMES[midi_note % 12] + str(midi_note // 12 - 1)  # Corrected the octave offset


In [ ]:
#this function constructs the chord structure for each type of chord named.

def generate_chords(start, end, chord_type='', num_notes=3):
    if chord_type not in ['major', 'minor', 'dom7', 'diminished', 'augmented', 'sus2', 'sus4']:
        raise ValueError('Invalid chord type. Must be one of "major", "minor", "dom7", "diminished", "augmented", "sus2", "sus4".')

    chords = []
    for root in range(start, end+1):
        if chord_type == 'major':
            chord = [root, root + 4, root + 7]  # Major triad
            if num_notes >= 4:
                chord.append(root + 11)  # Major seventh
            if num_notes >= 5:
                chord.append(root + 14)  # Major ninth
        elif chord_type == 'minor':
            chord = [root, root + 3, root + 7]  # Minor triad
            if num_notes >= 4:
                chord.append(root + 10)  # Minor seventh
            if num_notes >= 5:
                chord.append(root + 14)  # Minor ninth
        elif chord_type == 'dom7':
            chord = [root, root + 4, root + 7, root + 10]  # Dominant 7th
            if num_notes >= 5:
                chord.append(root + 14)  # Dominant 9th
        elif chord_type == 'diminished':
            chord = [root, root + 3, root + 6]
        elif chord_type == 'augmented':
            chord = [root, root + 4, root + 8]
        elif chord_type == 'sus2':
            chord = [root, root + 2, root + 7]
        elif chord_type == 'sus4':
            chord = [root, root + 5, root + 7]
        chords.append(chord)
    return chords

In [ ]:
start_note = 21  # A0
end_note = 108  # C8

#this is the dictionary of all the chords we want to generate. The key is the name of the chord, and the value is the chord structure.
chord_types = {
    'major': [3, 4, 5],
    'minor': [3, 4, 5],
    'dom7': [4, 5],
    'diminished': [3],
    'augmented': [3],
    'sus2': [3],
    'sus4': [3]
}

chords = {}

for chord_type, num_notes_list in chord_types.items():
    for num_notes in num_notes_list:
        key = f"{chord_type}_{'triads' if num_notes == 3 else 'sevenths' if num_notes == 4 else 'ninths'}"
        chords[key] = generate_chords(start_note, end_note, chord_type=chord_type, num_notes=num_notes)

# Now you can access any chord using the chords dictionary. For example:
print(chords['major_triads'])
print(chords['minor_sevenths'])

For this part of the code you'll need to download something called a soundfont. The soundfont is a file that contains the sounds of the piano. The soundfont is used to generate the audio files.You can find them free online. 

I provide links to the soundfonts used here in the README of the dataset. Hopefully it doesn't give you a virus haha!

In [ ]:
def play_chord(chord, chord_name, directory=''):
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    for i, chord in enumerate(chord):
        # Create a new MIDI file
        mid = MidiFile()
        track = MidiTrack()
        mid.tracks.append(track)

        # Change to a different instrument (0 = Acoustic Grand Piano)
        track.append(Message('program_change', program=0, time=0))

        # Add the notes of the chord
        for note in chord:
            track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=1920))
        for note in chord:
            track.append(Message('note_off', note=note, velocity=64, time=0))

        # Extract the root note name and its MIDI number
        root_note_name = get_note_name(chord[0])
        root_midi_number = chord[0]

        # Save the MIDI file with a descriptive name
        midi_filename = os.path.join(directory, f'{chord_name}_{root_note_name}_{root_midi_number}.mid')
        mid.save(midi_filename)

        # Convert the MIDI file to a WAV file
        wav_filename = os.path.join(directory, f'{chord_name}_{root_note_name}_{root_midi_number}.wav')
        fs.midi_to_audio(midi_filename, wav_filename)

        # Clean up the MIDI file
        os.remove(midi_filename)



In [ ]:
# Define the soundfonts and their paths. The name is important because it'll be used in the file names.
soundfonts = {
    'ES100':    '~/Downloads/soundfonts/_GD__Kawai_ES100_Concert_Grand_1__V3_/[GD] Kawai ES100 Concert Grand 1 [V3].sf2',
    'D274' :    '~/Downloads/soundfonts/_GD__Steinway_Model_D274/[GD] Steinway Model D274.sf2',
    'ES':       '~/Downloads/soundfonts/ES6_Pianos/ES6 Pianos.sf2',
    'Nord' :    '~/Downloads/soundfonts/NordGrand1/NordGrand1.sf2',
    'MODX8' :   '~/Downloads/soundfonts/Yamaha_MODX8_piano_fix/Yamaha MODX8 piano.sf2',
    # Add other soundfonts here as needed
}

# Define the chord types, their variable names, directory names, and chord names
chord_definitions = {
    'major_triads': {
        'var_name': 'major_triads', #this is the variable name of the chord type in the chords dictionary
        'dir_name': 'major_triad', #this is the directory name where the chords will be saved
        'chord_name': 'majortriad' #this is what it'll be named in the directory (IMPORTANT FOR LABELING, so be sure to give it a nice name :D)
    },
    'major_sevenths': {
        'var_name': 'major_sevenths',
        'dir_name': 'major_seventh',
        'chord_name': 'majorseventh'
    },
    'major_ninths': {
        'var_name': 'major_ninths',
        'dir_name': 'major_ninth',
        'chord_name': 'majorninth'
    },
    'minor_triads': {
        'var_name': 'minor_triads',
        'dir_name': 'minor_triad',
        'chord_name': 'minortriad'
    },

    'minor_sevenths': {
        'var_name': 'minor_sevenths',
        'dir_name': 'minor_seventh',
        'chord_name': 'minorseventh'
    },

    'minor_ninths': {
        'var_name': 'minor_ninths',
        'dir_name': 'minor_ninth',
        'chord_name': 'minorninth'
    },
    'dom7_sevenths': {
        'var_name': 'dom7_sevenths',
        'dir_name': 'dom7_seventh',
        'chord_name': 'dom7seventh'
    },

    'dom7_ninths': {
        'var_name': 'dom7_ninths',
        'dir_name': 'dom7_ninth',
        'chord_name': 'dom7ninth'
    },

    'diminished_triads': {
        'var_name': 'diminished_triads',
        'dir_name': 'diminished_triad',
        'chord_name': 'diminishedtriad'
    },

    'augmented_triads': {
        'var_name': 'augmented_triads',
        'dir_name': 'augmented_triad',
        'chord_name': 'augmentedtriad'
    },

    'sus2_triads': {
        'var_name': 'sus2_triads',
        'dir_name': 'sus2_triad',
        'chord_name': 'sus2triad'
    },

    'sus4_triads': {
        'var_name': 'sus4_triads',
        'dir_name': 'sus4_triad',
        'chord_name': 'sus4triad'
    },
    # Add other chord types here as needed
}

# Get the directory of the current script
script_directory = os.path.dirname(os.path.abspath(__file__))

# Set the base directory relative to the script's directory. Hopefully this works for u too
base_directory = os.path.join(script_directory, 'dataset', 'chords')

In [ ]:
for sf_name, sf_path in soundfonts.items():
    # Expand the ~ to the full path of the user's home directory and create FluidSynth instance
    fs_instance = FluidSynth(os.path.expanduser(sf_path))
    
    for chord_key, chord_info in chord_definitions.items():
        chord_var = globals()[chord_info['var_name']]
        directory = os.path.join(base_directory, chord_info['dir_name'])
        chord_name = f"{sf_name}_{chord_info['chord_name']}"
        
        play_chord(fs_instance, chord_var, chord_name, directory)